In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Reshape, Input, GlobalAveragePooling2D, LayerNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import   DenseNet121
from tensorflow.keras.models import Model

import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from  shutil import copyfile
import random

from sklearn.metrics import confusion_matrix, auc, roc_curve
import seaborn as sns

import tensorflow_addons as tfa
import cv2
import time

from keras import backend as K
from keras.models import Model
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Lambda
from keras.layers import add
from keras.layers import LeakyReLU


In [ ]:
def _normalize_img(img, label):
    mean, std = tf.math.reduce_mean(img), tf.math.reduce_std(img)
    img = (img - mean) / std
    #img = tf.cast(img, tf.float32) / 255.
    #img -= 127.5
    #img /= 128
    #img /= 127.5
    #img -= 1
    return (img, label)

def _resize_image(image):
    # Resize the image to a new size
    image = tf.image.resize(image,(160, 160))
    return image

<h1> Preparing Trainig data</h1>

In [ ]:
import pickle
res = 'E:\\Thesis\\deepface\\performance_results\\First_Repetition\\dataset_randomlySelected_4Flod_Repetition1_1sttrain_43.pickle'
with open(res, "rb") as f:
    flod_1 = pickle.load(f)
    

In [ ]:
labels_training = list()
for pic in flod_1:
    if "out" in pic:
        index = pic.split("\\")[-1].split("_")[0][1:] + "1"
        labels_training.append(int(index))
    else:
        index = pic.split("\\")[-1].split("_")[0][1:] + "0"
        labels_training.append(int(index))


In [ ]:
trainingSet_tensor = list(flod_1.values())

In [ ]:
list(flod_1.keys())[18000]

In [ ]:
labels_training[18000]

In [ ]:
print(len(labels_training))
print(len(trainingSet_tensor))


In [ ]:
flod_1 = None

<h1>New technology</h1>

In [ ]:
trainingDataset = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(trainingSet_tensor),tf.data.Dataset.from_tensor_slices(labels_training)))

In [ ]:
batch=512
trainingDataset = trainingDataset.shuffle(buffer_size=10000)
trainingDataset = trainingDataset.map(_normalize_img)

In [ ]:
trainingDataset = trainingDataset.batch(batch)
trainingDataset = trainingDataset.prefetch(tf.data.AUTOTUNE)
#trainingDataset = trainingDataset.prefetch(buffer_size=15)

In [ ]:
from deepface.basemodels import Facenet512,Facenet
model = Facenet.loadModel()
print(model.input_shape)
print(model.output_shape)
print(len(model.layers))

In [ ]:
for layer in model.layers[:-20]:
    layer.trainable = False 

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.layers import PReLU

# Replace ReLU activations with Leaky ReLU in the last 20 layers
for layer in model.layers[-20:]:
    if isinstance(layer, tf.keras.layers.Activation) and layer.get_config()['activation'] == 'relu':
        layer_index = model.layers.index(layer)
        
        # Create new LeakyReLU layer
        model.layers[layer_index].activation = PReLU(alpha_initializer="Zeros",name=layer.name)
        print(model.layers[layer_index].activation)
        print(layer_index)


In [ ]:
model.layers[438].get_config()

In [ ]:
lr=0.1
epsilon=1e-08
epochs=50

In [ ]:
lr_schedule= tf.keras.callbacks.LearningRateScheduler(lambda epoch:1e-4 * 10**(epoch / 20))
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr),  loss=tfa.losses.TripletSemiHardLoss(margin=0.2))

In [ ]:
history = model.fit(trainingDataset, epochs=epochs, callbacks=[lr_schedule, early_stop])

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
counter = 0
prefetch = "AUTOTUNE"


In [ ]:
h5_model = f"performance_results\\First_Repetition\\{counter}_Facenet_OptSGD{lr}_Batch{batch}_prefetch{prefetch}_Semihard_1stTrain.h5"
model.save(h5_model)


In [ ]:
loss = history.history['loss']
plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.plot(loss, label='Loss')
#plt.ylim(0,0)
plt.xlabel('epoch')
plt.savefig(f"performance_results\\First_Repetition\\Loss_Facenet_OptSGD{lr}_Batch{batch}_prefetch{prefetch}_Semihard_1stTrain.jpg")
plt.close()
plt.show()
counter += 1
